In [1]:
from parsing.input_layer_extractor import InputLayerExtractor
from utility.s3_mongo_inter import S3MongoInterface

import tensorflow as tf
import numpy as np

In [2]:
interface = S3MongoInterface()

# Returns an array of music for each new call, never ends (loops back to start)
data_source = interface.get_input_arrays({})

# def get_entry():
#     for entry in data_generator.next():
#         yield entry

In [3]:
def generate_data(trunc_backprop, single_y=False):
    depth = int(trunc_backprop)
    cur_data = data_source.next()
    for i in range(0, len(cur_data) - depth - 1, 1):
        X = cur_data[i:i+depth]
        if not single_y:
            y = cur_data[i+1:i+1+depth]
        else:
            y = cur_data[i+depth+1]
        yield X, y

def data_batch(batch_size, trunc_backprop, single_y=False):
    X, y = generate_data(trunc_backprop, single_y=single_y).next()
    for i in range(batch_size - 1):
        X_new, y_new = generate_data(trunc_backprop, single_y=single_y).next()
        X = np.vstack([X, X_new])
        y = np.vstack([y, y_new])
    yield X.reshape((batch_size, -1)), y.reshape((batch_size, -1))

def protected_data_batch(batch_size, trunc_backprop, single_y=False):
    try:
        yield data_batch(batch_size, trunc_backprop, single_y=single_y).next()
    except StopIteration:
        yield data_batch(batch_size, trunc_backprop, single_y=single_y).next()

In [ ]:
for _ in range(500):
    protected_data_batch(batch_size, truncated_backprop_length).next()

In [ ]:
X, y = data_batch(5, 50, single_y=True).next()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
len(list(y))

In [ ]:
np.reshape(y,)

In [ ]:
6400 / 128

In [4]:
truncated_backprop_length = 50
batch_size = 5
state_size = 10

input_size = 128
output_size = 128

num_epochs=500
num_layers = 3

In [5]:
# batch_size is number of songs it's looking at simulataneously
# total series length is total number of beats of all songs

# X is a complete song
X = tf.placeholder(
    tf.float32,
    [batch_size, truncated_backprop_length * input_size],
    name='X'
)  # num rows is num of beats for song (like a batch)
# roll forward by 1
y = tf.placeholder(
    tf.float32,
    [batch_size, input_size],  # Fixme?
    name='y'
)  # num rows is num of beats for song (like a batch)

init_state = tf.placeholder(
    tf.float32,
    [num_layers, 2, batch_size, state_size]
)

state_per_layer_list = tf.unstack(init_state, axis=0)
rnn_tuple_state = tuple(
    [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0,:,:], state_per_layer_list[idx][1,:,:])
    for idx in range(num_layers)]
)

W2 = tf.Variable(
    np.random.rand(state_size, output_size),
    dtype=tf.float32
)
b2 = tf.Variable(
    np.zeros((1, output_size)),
    dtype=tf.float32
)

# # X = tf.reshape(X, (5, 50, 128))
# inputs_series = tf.unstack(
#     tf.reshape(X, (5, 50, 128)),
#     axis=1
# )

# # inputs_series = tf.unstack(X, axis=0)
# labels_series = tf.unstack(
#     tf.reshape(y, (5, 50, 128)),
#     axis=1
# )

# lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(
#     state_size, state_is_tuple=True,
#     reuse=tf.get_variable_scope().reuse
# )
stacked_rnn = []
for _ in range(num_layers):
    stacked_rnn.append(tf.nn.rnn_cell.LSTMCell(
        state_size, state_is_tuple=True
    ))
lstm_cells = tf.nn.rnn_cell.MultiRNNCell(
    stacked_rnn,
    state_is_tuple=True
)

# 5 x 6400 (128 x 50) x 10
states_series, current_state = tf.nn.dynamic_rnn(
    lstm_cells,
    tf.expand_dims(X, -1),
    initial_state=rnn_tuple_state
)

In [22]:
lstm_cells.

In [6]:
states_series

<tf.Tensor 'rnn/transpose:0' shape=(5, 6400, 10) dtype=float32>

In [7]:
states_series = tf.reshape(states_series, [-1, state_size])

states_series

<tf.Tensor 'Reshape:0' shape=(32000, 10) dtype=float32>

In [34]:
W2.eval(session=sess)

NameError: name 'sess' is not defined

In [9]:
b2

<tf.Variable 'Variable_1:0' shape=(1, 128) dtype=float32_ref>

In [51]:
_current_state

NameError: name '_current_state' is not defined

In [52]:
len(current_state)

3

In [49]:
type(current_state[0])

tensorflow.python.ops.rnn_cell_impl.LSTMStateTuple

In [46]:
current_state[0][1]

<tf.Tensor 'rnn/while/Exit_3:0' shape=(5, 10) dtype=float32>

In [32]:
type(current_state[0])

tensorflow.python.ops.rnn_cell_impl.LSTMStateTuple

In [10]:
logits = tf.matmul(states_series, W2) + b2

In [11]:
logits

<tf.Tensor 'add:0' shape=(32000, 128) dtype=float32>

In [12]:
y

<tf.Tensor 'y:0' shape=(5, 128) dtype=float32>

In [13]:
logits = tf.reshape(
    logits,
    [batch_size, input_size * truncated_backprop_length, input_size],
)

In [14]:
logits

<tf.Tensor 'Reshape_1:0' shape=(5, 6400, 128) dtype=float32>

In [15]:
labels = tf.reshape(y, [-1])

In [16]:
labels = y

In [17]:
labels

<tf.Tensor 'y:0' shape=(5, 128) dtype=float32>

In [18]:
logits_series = tf.unstack(logits, axis=1)

In [19]:
logits_series

[<tf.Tensor 'unstack_1:0' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:1' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:2' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:3' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:4' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:5' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:6' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:7' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:8' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:9' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:10' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:11' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:12' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:13' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:14' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:15' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:16' shape=(5, 128) dtype=float32>,
 <tf.Tensor 'unstack_1:1

In [20]:
predictions_series = [
    tf.nn.sigmoid(logits) for logits in logits_series
]

In [ ]:
predictions_series

In [ ]:
# Merges first two dimensions whatevs x 10
states_series = tf.reshape(states_series, [-1, state_size])


logits = tf.matmul(states_series, W2) + b2
labels = tf.reshape(y, [-1])

logits_series = tf.unstack(
    tf.reshape(
        logits,
        [batch_size, truncated_backprop_length, input_size]
    ),
    axis=1
)
predictions_series = [
    tf.nn.sigmoid(logits) for logits in logits_series
]

# logits_series = [
#     tf.matmul(state, W2) + b2 for state in states_series
# ]
# predictions_series = [
#     tf.nn.sigmoid(logits) for logits in logits_series
# ]



In [21]:
losses = tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits, labels=labels
)
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [ ]:

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_list = []
    
    for epoch in range(num_epochs):
        # Initialize vars
        X_, y_ = protected_data_batch(
            batch_size, truncated_backprop_length, single_y=True
        ).next()
#         _current_cell_state = np.zeros((batch_size, state_size))
        _current_state = np.zeros((num_layers, 2, batch_size, state_size))
#         _current_hidden_state = np.zeros((batch_size, state_size))
        # One step
        _total_loss, _train_step, _current_state, _predictions_series = sess.run(
            [total_loss, train_step, current_state, predictions_series],
            feed_dict={
                X: X_,
                y: y_,
                init_state: _current_state

            })
#         _current_cell_state, _current_hidden_state = _current_state
        loss_list.append(_total_loss)
        
        if epoch % 100 == 0:
            print("Epoch", epoch, "loss:", _total_loss)

In [ ]:
loss_list

In [ ]:
len(predictions_series)

In [47]:
tf.nn.rnn_cell.MultiRNNCell()

0
1
